# Waveform Generator (all trials)

In [11]:
import numpy as np
import math
import json
import random

def waveforms(N_A, N_g, N_f, t0_tf, T, B, trials, seedn=1, inputfile="input", 
              phi0=0, A0=1, Af=50, g0=0, gf=2, F0=90, Ff=110, N=10000):

    # initalizes the arrays which span parameter space, and their lengths
    A_RANGE=np.linspace(A0,Af,N_A)
    G_RANGE=np.linspace(g0,gf,N_g)
    F_RANGE=np.linspace(F0,Ff,N_f)

    # number of parameters available
    A_LEN, G_LEN, F_LEN = N_A, N_g, N_f
    
    waveform_data={}
    for j in range(trials):
        waveform_data.update({j:[[],[]]})
        
        # calculates random indice for each parameter (A, f, g)
        A_RAN=random.randint(0,A_LEN-1)
        G_RAN=random.randint(0,G_LEN-1)
        F_RAN=random.randint(0,F_LEN-1)
        
        # calculates random parameters A, f, g
        A, gamma, f = A_RANGE[A_RAN], G_RANGE[G_RAN], F_RANGE[F_RAN]
        
        dt=T/N # time resolution

        t0=(T-t0_tf)*np.random.random(1)[0]  # randomly generate start time
        START_INDEX=math.floor(t0/dt)        # find index associated with time

        ##NOTE: using 't0' instead of some multiple of dt may cause issues later
        
        SIG_LEN = (math.floor(t0_tf/dt)+1 if (t0_tf != T) else N) # calculate # of indexes signal takes
        INJECTED = np.zeros(N)                 # initalize injected signal, with N size array of zeroes
        for i in range(SIG_LEN):
            INJECTED[START_INDEX + i]=t0+i*dt       # fill in injected signal with its time stamps

        w = 2 * np.pi * f
        
        # replace timestamps with their displacement values
        SR = INJECTED[START_INDEX : START_INDEX+SIG_LEN][:]
        INJECTED[START_INDEX : START_INDEX+SIG_LEN] = A*np.sin(w*(SR-t0) + phi0)*np.exp(-gamma*(SR-t0))
        
        # Purposed for Quadrature Sum
        D_i = [] # list of each differently seeded waveform
        for n in range(seedn):
            np.random.seed(seed = n)
            NOISE = np.random.normal(scale=(B/(math.sqrt(3)*2)), size=N)  # Noise!
            D_i.append(list(NOISE + INJECTED))  # complete data!
        
        # gets parameters and data for each trial, stuffs it into dictionary
        parameters = [A, f, gamma, t0]
        waveform_data[j][0], waveform_data[j][1] = parameters, D_i
        
    # each trial has list of parameters used and list of data values
    with open("{}-waveform_data.json".format(inputfile) , "w") as f:
        json.dump(waveform_data, f, indent=2, sort_keys=True)

In [41]:
#  N_A, N_g, N_f, t0_tf, T, B, trials
waveforms(4, 4, 4, 4, 10, 0, 10, N=250, seedn=1)

# Stat Analysis (per trial)

In [66]:
import json
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import math
import time

# Produces a template given a position in parameter space
def template(A, f, gamma, duration, dt):
    t = np.arange(0, duration + dt, dt)
    w = 2 * np.pi * f
    return A * np.sin(w*t)*np.exp(-gamma*t)

# Produces a cross corelation function given a data input and a template in parameter space
def CrossCorrelation(data, template, dt):
    ii = 0
    M = []

    while len(data[ii:]) >= len(template):
        M.append(np.sum((data[ii: len(template) + ii] * template)))
        ii+=1

    return M

# Produces chi square at each "slide"
def ChiSquare(data, template, dt):
    ii = 0
    C = []

    while len(data[ii:]) >= len(template):
        C.append(-1 * np.sum((data[ii: len(template) + ii] - template) ** 2))
        ii += 1

    return C

def modulator(rho_ij, D, dt):

    rho_mod_D, RHO_ij = [] , rho_ij[:]

    dn , L = math.floor(2*D/dt) , len(RHO_ij)

    for i in range(0,L-(L%dn),dn):
        rho_mod_D.append(max(RHO_ij[i:i+dn]))

    if (L-(L%dn)) != L :
        rho_mod_D.append(max(RHO_ij[L-(L%dn):L]))

    return rho_mod_D

def stat_analysis(trialn, D, N_A, N_g, N_f, t0_tf, T, trials, thresholds = False, 
                  essentials=False, seedn=1, inputfile="input", A0=1, Af=50, g0=0, 
                  gf=2, F0=90, Ff=110, N_t=10000):

    # initalizes the arrays which span parameter space, and their lengths
    A_RANGE=np.linspace(A0,Af,N_A)
    G_RANGE=np.linspace(g0,gf,N_g)
    F_RANGE=np.linspace(F0,Ff,N_f)

    A_LEN, G_LEN, F_LEN = N_A, N_g, N_f

    # constructs timestep resolution, and saves N and t0/tf internally 
    N, dt = N_t, T/N_t

    # constructs time range to pick injected signal start time from/ corresponding length 
    t_RANGE=np.linspace(0,T-(t0_tf),int(N_t*(1-((t0_tf)/(T)))))
    t_LEN=len(t_RANGE)

    # initialize arrays for various data/cross-correlations/chi-squares 
    noise = []

    # constructs all templates which correspond to points in the parameter space
    TEMPLATES_AFG=[ template( A, f, g, t0_tf, dt) for A in A_RANGE 
                   for g in G_RANGE for f in F_RANGE]

    AFG_PAIR=[ [A, f, g] for A in A_RANGE 
                   for g in G_RANGE for f in F_RANGE]

    # Reads waveform data file 
    with open("{}-waveform_data.json".format(inputfile),"r") as f: 
        waveform_data = json.load(f)

    waveform_data = waveform_data[str(trialn)] # trialn's parameters and differently seeded data
    
    output={}

    output.update({trialn:[[],[],[],[]]})

    # isolates random a-g-f pair / data set 
    temp_AGFT, data = waveform_data[0], waveform_data[1]

    noise.append(data) 

    output[trialn][0], output[trialn][1] = temp_AGFT, data  # stores random a-g-f pair / data set 

    Quad_CRS = []
    Quad_CHI = []
    
    # performs base static calculation across parameter space
    # Quadrature Sum
    for n in range(seedn): # Use seedn as index for data

        CRS_COR, CHI_SQR = [[],[]]

        for temp in TEMPLATES_AFG:

            CC_dh = list(CrossCorrelation(data[n], temp, dt))
            CRS_COR.append(CC_dh)

            CS_dh = list(ChiSquare(data[n], temp, dt))
            CHI_SQR.append(CS_dh)

        Quad_CRS.append(CRS_COR) # now a 3d list of seedn statistics, with 2d list statistics per waveform
        Quad_CHI.append(CHI_SQR)

    CRS_COR = np.sum(np.array(Quad_CRS) ** 2, axis = 0) ** .5 # Quadrature sum of each seed's statistic
    CHI_SQR = np.sum(np.array(Quad_CHI) ** 2, axis = 0) ** .5

    # stores base statistics to attribute
    cross_cor = CRS_COR
    chi = CHI_SQR
    output[trialn][2], output[trialn][3] = CRS_COR, CHI_SQR # store quadrature summed based statistics
    
#calculates test statistic, stores it internally,
#and returns a copy of it as a dictionary 
# trying connect stat and anlysis function
    
    # counts number of tempates in parameter space
    PSPACE_LEN = len(AFG_PAIR)
    
    # String to equation!
    stats = ["CC_IJ","CS_IJ","CC_IJ/abs((1+CS_IJ))"]

    # initalizes rho statistic dictionary/stat outputs collector
    RHO = {}
    stat_outputs = []
    
    for stat in stats:

        # indexed to loops through parameter space templates and
        # calculates each rho_ij given template j
        rho_i = []
        for j in range(PSPACE_LEN):
            CC_IJ = np.array(cross_cor[j][:])
            CS_IJ = np.array(chi[j][:])

            # Evaluates string (Exec gave issues... eval is the same concept though)
            p = eval(stat)
            rho_i.append(list(p))
        stat_outputs.append(rho_i) # stat_outputs is 3d list holding a 2d list of a stat's outputs for each template
    RHO.update({ trialn : stat_outputs })

    if (2*D >= dt):
        
        # gets the length of linearized template space
        TEMP_LEN=len(cross_cor) # number of templates 

        RHO_MOD={}
        MAX_OS={}
        MAX_BG_TEMP={}
        pot_thresholds = [[],[],[]]
        BG_VALS_IJ, FG_VAL_IJ = [], []
        
        # seperates fg value from bg value
        T0_2D = math.floor(output[trialn][0][3]/(2*D))

        for j in range(TEMP_LEN):
            MAX_BG_TEMP.update({ j : list(np.zeros(len(stats)))})

        for stat in range(len(stats)):

            BG_VALS_ij, FG_VAL_ij = [], []
            for j in range(TEMP_LEN):

                # calculates bg values + fg values
                BG_VALS_ij.append(modulator(RHO[trialn][stat][j][:],D,dt))

                FG_VAL_ij.append(BG_VALS_ij[j].pop(T0_2D))
                    
            BG_VALS_IJ.append(BG_VALS_ij) # 3d list
            FG_VAL_IJ.append(FG_VAL_ij) # 2d list
            
            for j in range(TEMP_LEN):
                new_max=max(RHO_MOD[trialn][0][stat][j]) # max ofsource peak for each template

                if (MAX_BG_TEMP[j][stat] < new_max): # every value in MAX_BG_TEMP dictionary changes from 0 to that templates max
                    MAX_BG_TEMP[j][stat] = new_max
                
        pot_thresholds += FG_VAL_IJ # 3d, of 2d lists per trial, of 1d lists per stat BAD!
        RHO_MOD.update({ trialn: [ BG_VALS_IJ, FG_VAL_IJ ] }) # these are the peaks we look for
        MAX_OS.update({ trialn: [tuple(output[trialn][0][0:3]),max(RHO_MOD[trialn][1][stat])] }) # stores t0 and forground peaks for each trial
        

    else: 
        print("invalid D; it is required that 2*D >= T/N")

    # output holds trialn's parameters, data, cross-correlation, chi-square
    with open("output_{}.json".format(trialn), "w") as f: 
        json.dump(output, f, indent=2, sort_keys=True)

    # RHO_MOD holds trialn's background and forground values
    with open("Peaks_{}.json".format(trialn), "w") as f:
        json.dump(RHO_MOD, f, indent=2, sort_keys=True)

    # MAX_OS holds trialn's max onsource peak
    with open("Max_OS_{}.json".format(trialn), "w") as f:
        json.dump(MAX_OS, f, indent=2, sort_keys=True)

    # MAX_BG_TEMP holds trialn's "new_max" for each statistic used
    with open("Max_BG_TEMP{}.json".format(trialn), "w") as f: # Merged
        json.dump(MAX_BG_TEMP, f, indent=2, sort_keys=True)

# You want bottom two if statements to be False for the first trial run
# then change to True for the rest of the instances

    # pot_thresholds holds trialn's forground peaks
    if thresholds == False: # False means to start the threshold file that will hold the pot_thresholds
        with open("thresholds.json", "w") as f:
            json.dump(pot_thresholds, f, indent=2, sort_keys=True)
    else:
        with open("thresholds.json", "r") as f:
            current_thresholds = json.load(f)
            
        pot_thresholds += current_thresholds
        
        with open("thresholds.json", "w") as f:
            json.dump(pot_thresholds, f, indent=2, sort_keys=True)
    
    # essentials holds all the values that don't change for each trialn run
    if essentials == False:
        essent = {"essentials":[A_LEN,F_LEN,G_LEN,F_RANGE,A_RANGE,G_RANGE,AFG_PAIR,trials]}
        with open("essentials.json", "w") as f:
            json.dump(essent, f, indent=2, sort_keys=True)

# Argparse must be added
# Bash script to run nohup code

In [67]:
# stat_analysis(trialn,D, N_A, N_g, N_f, t0_tf, T, trials, seedn = 1, stat=None, 
#                   inputfile="input", A0=1, Af=50, g0=0, gf=2, F0=90, Ff=110, N_t=10000):
    
stat_analysis(1, .02, 4, 4, 4, 4, 10, 10, N_t=250, seedn=1)

KeyError: 1

# Json Combiner (all trials)

In [8]:
import glob
import json

def json_combiner(jsons_path, merge_path_name):
    
    # Include last / at end of path!
    files = glob.glob("{}*.json".format(jsons_path))

    count = 0
    for file in files:
        if count == 0:
            count += 1
            with open(file, "r") as f:
                C_dictionary = json.load(f)
        else:
            with open(file, "r") as f:
                C_dictionary.update(json.load(f))

    with open('{}.json'.format(merge_path_name), 'w') as f:
        json.dump(json_list, f, indent=1)

# Plot Functions (All trials)

In [125]:
def test_plotter(T, N, var1=0, var2=0,bg_test=True):
# essent = {"essentials":[A_LEN,F_LEN,G_LEN,F_RANGE,A_RANGE,G_RANGE,AFG_PAIR,trials]}
    with open("Merged_essentials.json", "r") as f:
        essentials = json.load(f)
    A_LEN = essentials["essentials"][0]
    F_LEN = essentials["essentials"][1]
    G_LEN = essentials["essentials"][2]
    F_RANGE = essentials["essentials"][3]
    A_RANGE = essentials["essentials"][4]
    G_RANGE = essentials["essentials"][5]
    AFG_PAIR = essentials["essentials"][6]
    trials = essentials["essentials"][7]

    with open("Merged_output.json", "r") as f:
        output = json.load(f)
    
    with open("Merged_Peak.json", "r") as f:
        RHO_MOD = json.load(f)

    # Calculates the index of the attribute variable
    # (A, F, or G) which the heat map should marginalize
    # over
    if ((var1 <= 2) and (var2 <= 2)):    #checks if range valid
        if ((0<= var1) and (0<= var2)):  #checks if range valid
            if (var2 < var1):            #properly orders
                var1, var2 = var2, var1
                index=(1/4)*(var1-1)*(var1-2)*(var2)*(7-3*var2) #maps to needed index

            elif (var2==var1):
                index=var2  # if the values are the same, set as index

            else:
                index=(1/4)*(var1-1)*(var1-2)*(var2)*(7-3*var2) #maps to needed index

        else:
            print("improper index inputs: both must be >= 0")
            index=0
    else:
        print("improper index inputs: both must be <= 2")
        index=0

    mapping={}   #initalizes mapping for heat map

    holder=np.full((A_LEN, F_LEN, G_LEN), 0, dtype=list)

    # builds link between keys and np-array 
    for j in range(F_LEN):
        for i in range(A_LEN):
            for k in range(G_LEN):
                F=F_RANGE[j]
                A=A_RANGE[i]
                G=G_RANGE[k]

                holder[(i,j,k)]=[0]
                mapping.update({ (A,F,G) : holder[(i,j,k)]})

    #initalizes threshold axis points, counts for each threshold
    #and space of parameter space
    THRS_AXIS=[]
    COUNT_AXIS=[]
    PSPACE_LEN=len(AFG_PAIR)
    PSACE_PAIRS=[]

    #sets current threshold value
    # how many templates succeeded per threshold (succ_count_thrhld), how many trials succeeded per threshold (heatcount)
    for thrshld in np.linspace(0,T,N):

        THRS_AXIS.append(thrshld)
        succ_count_thrhld=0

        #loops from trial/ parameter space pairs 
        for i in range(trials):
            heat_count=0
            for j in range(PSPACE_LEN):

                RM_ij=np.array(RHO_MOD[i][0][j])
                FG_ij=RHO_MOD[i][1][j]
                N_BG_ij=len(RM_ij[ RM_ij > FG_ij])

                #tests if the given pair passes the threshold test
                if (bg_test==True):
                    if ((N_BG_ij==0) and (FG_ij > thrshld)): # (per template) offsource peaks < onsource peak, onsource peak > threshold

                        if heat_count==0:
                            succ_count_thrhld+=1
                            mapping[tuple(output[i][0][:3])][0]+=1 # adds 1 to that paramater combo's value, default is 0 
                            heat_count+=1
                else:
                    if (FG_ij > thrshld): # (per template) onsource peak > threshold

                        if heat_count==0:
                            succ_count_thrhld+=1
                            mapping[tuple(output[i][0][:3])][0]+=1 # adds 1 to that paramater combo's value, default is 0 
                            heat_count+=1

        COUNT_AXIS.append(succ_count_thrhld)

    plt.plot(THRS_AXIS,COUNT_AXIS)
    plt.xlabel("$Threshold$")
    plt.ylabel("Counts")
    plt.show()

    #redefines things to be normal arrays inside
    for j in range(F_LEN):
        for i in range(A_LEN):
            for k in range(G_LEN):
                holder[(i,j,k)]=holder[(i,j,k)][0] # I think [0] is making the list value for each parameter combo just its value

    #marginalizes array in direction of index
    index=int(index) # index is still weird
    w=holder.sum(index)

    #builds a copy of the original array, but fixes things to be integers 
    cop=np.full(w.shape,0) # array in shape of marginalized array of holder values
    z=[(i,j) for i in range(w.shape[0]) for j in range(w.shape[1])] 
    for tup in z: # z holds the index for cop, these indexes are each of two variables in use
        cop[tup]=int(w[tup])

    label=np.array(["amplitude","frequency", "gamma"])
    x=np.array([0, 1, 2])
    x=x[ x != index] # parameter index that aren't index are used

    plt.imshow( cop, cmap=plt.cm.hot)
    plt.xlabel(label[x[1]])
    plt.ylabel(label[x[0]])

    #plt.imshow( heat_array, cmap=plt.cm.hot) 

def ROC_Data(T0, Tf, N):

    with open("Merged_essentials.json", "r") as f:
        essentials = json.load(f)
    A_LEN = essentials["essentials"][0]
    F_LEN = essentials["essentials"][1]
    G_LEN = essentials["essentials"][2]
    F_RANGE = essentials["essentials"][3]
    A_RANGE = essentials["essentials"][4]
    G_RANGE = essentials["essentials"][5]
    AFG_PAIR = essentials["essentials"][6]
    trials = essentials["essentials"][7]
    
    with open("Merged_Peak.json", "r") as f:
        RHO_MOD = json.load(f)

    PSPACE_LEN=len(AFG_PAIR)

    # Stats per threshold
    Detection_Prob = []
    New_False_Prob = []

    #sets current threshold value
    for thrshld in np.linspace(T0,Tf,N):

        # Detection/ False Alarm probability counters
        Detect_count = 0
        False_count = 0

        #loops from trial/ parameter space pairs 
        for i in range(trials):

            # Detection Probability
            Max_FG_ij = max(RHO_MOD[i][1]) # max of onsources per trial
            if Max_FG_ij > thrshld:
                Detect_count += 1

            for j in range(PSPACE_LEN):

                RM_ij=np.array(RHO_MOD[i][0][j])
                FG_ij=RHO_MOD[i][1][j]
                N_BG_ij=len(RM_ij[ RM_ij > FG_ij])

                # False Alarm probability
                falses_ij = len(RM_ij[RM_ij > thrshld])
                False_count += falses_ij

        # Detection/False Alarm probability stats
        Detect_stat = Detect_count / trials
        False_stat = False_count / (len(RHO_MOD[i][0][0]) * PSPACE_LEN * trials)

        # Appending stat per threshold
        Detection_Prob.append(Detect_stat)
        New_False_Prob.append(False_stat)
        
        return(Detection_Prob, New_False_Prob)

def ROC_Curve(n_s, N, new_stat_only=True, outputfile="ROC_test"):

    with open("Merged_Peak.json", "r") as f:
        RHO_MOD = json.load(f)

    with open("Merged_thresholds.json", "r") as f:
        thresholds = json.load(f)
    
    # Choose soley n-rho's or an addition stat
    if new_stat_only == True:
        stat_list = []
    else:
        stat_list = ["CC_IJ","CS_IJ"]

    # different ^n rhos are appended
    for n in range(1,n_s+1):
        rho = "CC_IJ/((1+abs(CS_IJ))**{})".format(n)
        stat_list.append(rho)

    stat_length = len(stat_list)
    for s in range(stat_length):

        rho(stat = stat_list[s])
        window(.02)
        Detection_Prob, New_False_Prob = ROC_Data(min(thresholds), max(thresholds), N)
        plt.plot(New_False_Prob, Detection_Prob, label=stat_list[s])

    plt.xlabel("New_False_Probs")
    plt.ylabel("Detection_Probs")
    plt.title("ROC Curve:n's={}:N={}".format(n_s,N))
    plt.legend()
    plt.savefig("{}-ROC.png".format(outputfile))
    plt.show()

def Scatter_plotter(thrshld, xvar, yvar): 

    with open("Merged_essentials.json", "r") as f:
        essentials = json.load(f)
    A_LEN = essentials["essentials"][0]
    F_LEN = essentials["essentials"][1]
    G_LEN = essentials["essentials"][2]
    F_RANGE = essentials["essentials"][3]
    A_RANGE = essentials["essentials"][4]
    G_RANGE = essentials["essentials"][5]
    AFG_PAIR = essentials["essentials"][6]
    trials = essentials["essentials"][7]

    with open("Merged_output.json", "r") as f:
        output = json.load(f)

    label=["amplitude","frequency", "gamma"]
    PSPACE_LEN=len(AFG_PAIR)
    SUCC_PAIRS=([],[])
    FAIL_PAIRS=([],[])

    #loops from trial/ parameter space pairs 
    for i in range(trials):

        succ_count_thrhld=0  #test condition for adding to fail array

        for j in range(PSPACE_LEN):

            RM_ij=np.array(RHO_MOD[i][0][j])  #get moded rho_ij background values
            FG_ij=RHO_MOD[i][1][j]            #get moded rho_ij foreground value
            N_BG_ij=len(RM_ij[ RM_ij > FG_ij])     # get background vals > froeground vals

            #tests if the given pair passes the threshold test
            if ((N_BG_ij==0) and (FG_ij > thrshld)):

                parameter_x=output[i][0][xvar]  #get parameter 'xvar' of trial i
                parameter_y=output[i][0][yvar]  #get parameter 'yvar' of trial i
                SUCC_PAIRS[0].append(parameter_x) #add to successes
                SUCC_PAIRS[1].append(parameter_y)
                succ_count_thrhld+=1
                break

        #sees if trial i failed the test; adds parameters to fail array
        if succ_count_thrhld==0:
            parameter_x=output[i][0][xvar]
            parameter_y=output[i][0][yvar]
            FAIL_PAIRS[0].append(parameter_x)
            FAIL_PAIRS[1].append(parameter_y)

    colors = ("red", "blue")
    groups = ("pass", "fail")
    marks = ("o", "*")
    data = ( SUCC_PAIRS, FAIL_PAIRS )

    # Create plot
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    for data, marks, color, group in zip(data, marks, colors, groups):
        x , y = data
        ax.scatter(x, y, marker=marks, c=color, edgecolors='none', s=30, label=group)

    plt.title('found-missed plot')
    plt.xlabel(label[xvar])
    plt.ylabel(label[yvar])
    plt.legend(loc=2)
    plt.show()

def index_counter(var1,var2):
    
    # Calculates the index of the attribute variable
    # (A, F, or G) which the heat map should marginaliz 
    # over
    if ((var1 <= 2) and (var2 <= 2)):    #checks if range valid
        if ((0<= var1) and (0<= var2)):  #checks if range valid
            if (var2 < var1):            #properly orders
                var1, var2 = var2, var1
                index=(1/4)*(var1-1)*(var1-2)*(var2)*(7-3*var2) #maps to needed index

            elif (var2==var1):
                index=var2  # if the values are the same, set as index

            else:
                index=(1/4)*(var1-1)*(var1-2)*(var2)*(7-3*var2) #maps to needed index

        else:
            print("improper index inputs: both must be >= 0")
            index=0
    else:
        print("improper index inputs: both must be <= 2")
        index=0

    return index

def heatmap(var1, var2, T, max_OS=-1, max_BG=-1):

    with open("Merged_essentials.json", "r") as f:
        essentials = json.load(f)
    A_LEN = essentials["essentials"][0]
    F_LEN = essentials["essentials"][1]
    G_LEN = essentials["essentials"][2]
    F_RANGE = essentials["essentials"][3]
    A_RANGE = essentials["essentials"][4]
    G_RANGE = essentials["essentials"][5]
    AFG_PAIR = essentials["essentials"][6]
    trials = essentials["essentials"][7]

    with open("Merged_output.json", "r") as f:
        output = json.load(f)
    
    with open("Merged_Max_OS.json", "r") as f:
        Max_OS = json.load(f)

    with open("Merged_Max_BG_TEMP.json", "r") as f:
        Max_BG_TEMP = json.load(f)
    
    index=index_counter(var1,var2)

    mapping={}   #initalizes mapping for heat map
    count_mapping={} #initalies array to count occurances of certain parameters
    BG_mapping={}

    holder=np.full((A_LEN, F_LEN, G_LEN), 0, dtype=list)
    count_holder=np.full((A_LEN, F_LEN, G_LEN), 0, dtype=list)
    BG_holder=np.full((A_LEN, F_LEN, G_LEN), 0, dtype=list)

    # builds link between keys and np-array 
    for j in range(F_LEN):
        for i in range(A_LEN):
            for k in range(G_LEN):
                F=F_RANGE[j]
                A=A_RANGE[i]
                G=G_RANGE[k]

                holder[(i,j,k)]=[0]
                count_holder[(i,j,k)]=[0]
                BG_holder[(i,j,k)]=[0]

                mapping.update({ (A,F,G) : holder[(i,j,k)]})
                count_mapping.update({ (A,F,G) : count_holder[(i,j,k)]})
                BG_mapping.update({ (A,F,G) : BG_holder[(i,j,k)]})

    #form BG mappings
    num_j=0
    for j in AFG_PAIR:
        BG_mapping[tuple(j)][0]=MAX_BG_TEMP[num_j]
        num_j+=1

    # form OS mappings
    for OS in MAX_OS.values():
        if (T<=OS[1]):
            mapping[OS[0]][0]+=float(OS[1])
            count_mapping[OS[0]][0]+=1

    #redefines things to be normal arrays inside 
    for j in range(F_LEN):
        for i in range(A_LEN):
            for k in range(G_LEN):
                holder[(i,j,k)]=holder[(i,j,k)][0]
                count_holder[(i,j,k)]=count_holder[(i,j,k)][0]
                BG_holder[(i,j,k)]=BG_holder[(i,j,k)][0]

    #marginalizes array in direction of index
    index=int(index)
    plot_array=holder.sum(index)
    normalizer=count_holder.sum(index)
    BG_plot=BG_holder.sum(index)

    #builds a copy of the original array, but fixes things to be integers 
    pa_cop=np.full(plot_array.shape,0.0)
    n_cop=np.full(normalizer.shape,0.0)
    BG_cop=np.full(BG_plot.shape,0.0)
    z=[(i,j) for i in range(plot_array.shape[0]) for j in range(plot_array.shape[1])]
    for tup in z:
        pa_cop[tup]=plot_array[tup]
        n_cop[tup]=normalizer[tup]
        BG_cop[tup]=BG_plot[tup]

    #normalize
    pa_cop[n_cop>0]=pa_cop[n_cop>0]/n_cop[n_cop>0]

    if (max_OS==-1):
        vmax_val=pa_cop.max()
    else:
        vmax_val=max_OS

    label=np.array(["A", "F", "G"])
    IL=np.array([0, 1, 2])
    IL=IL[ IL != index]

    XV="self."+label[IL[1]]+"_RANGE"
    XA=eval(XV)

    YV="self."+label[IL[0]]+"_RANGE"
    YA=eval(YV)

    contours = plt.contour(XA, YA, pa_cop, 10, colors='blue')
    plt.clabel(contours, inline=True, fontsize=8)

    plt.contourf(XA, YA, pa_cop, 100, cmap='hot', alpha=1, vmin=T, vmax=vmax_val);
    plt.colorbar();

    plt.xlabel(label[IL[1]])
    plt.ylabel(label[IL[0]])
    plt.title('Max On Source; t='+str(T))
    plt.show()

    if (max_BG==-1):
        vmax_val=BG_cop.max()
    else:
        vmax_val=max_BG

    contours = plt.contour(XA, YA, BG_cop, 10, colors='blue')
    plt.clabel(contours, inline=True, fontsize=8)

    plt.contourf(XA, YA, BG_cop, 100, cmap='hot', alpha=1, vmin=T, vmax=vmax_val);
    plt.colorbar();

    plt.xlabel(label[IL[1]])
    plt.ylabel(label[IL[0]])
    plt.title('Max Background; t='+str(T))
    plt.show()


# JSON Options:

In [7]:
# All jsons to open (post-combining)

with open("Merged_essentials.json", "r") as f:
    essentials = json.load(f)
A_LEN = essentials["essentials"][0]
F_LEN = essentials["essentials"][1]
G_LEN = essentials["essentials"][2]
F_RANGE = essentials["essentials"][3]
A_RANGE = essentials["essentials"][4]
G_RANGE = essentials["essentials"][5]
AFG_PAIR = essentials["essentials"][6]
trials = essentials["essentials"][7]

with open("Merged_output.json", "r") as f:
    output = json.load(f)

with open("Merged_Peak.json", "r") as f:
    RHO_MOD = json.load(f)

with open("Merged_Max_OS.json", "r") as f:
    Max_OS = json.load(f)

with open("Merged_Max_BG_TEMP.json", "r") as f:
    Max_BG_TEMP = json.load(f)

with open("Merged_thresholds.json", "r") as f:
    thresholds = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'Merged_essentials.json'